In [13]:
import os
import mne
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据处理函数
# 假设已经定义好数据处理函数 process_set_files 和 standardize_data

def process_set_files(set_files, words, tmin, tmax):
    all_X = []
    all_y_word = []
    for set_file in set_files:
        try:
            raw = mne.io.read_raw_eeglab(set_file, preload=True)
            print("文件读取成功！")
        except Exception as e:
            print(f"读取文件时出错: {e}")
            exit()

        # 去除不需要的通道
        channels_to_drop = [ch for ch in ['VEOG', 'HEOG', 'Trigger'] if ch in raw.ch_names]
        raw.drop_channels(channels_to_drop)

        # 滤波处理
        raw.filter(0.5, 80., fir_design='firwin')

        events, current_event_id_map = mne.events_from_annotations(raw)
        event_id = {}
        event_code_to_word = {}
        
        for i, word in enumerate(words):
            think_code = str(50 + i)  
            think_event_id = current_event_id_map.get(think_code)
            
            if think_event_id is not None:
                event_id[f'想_{word}'] = think_event_id
                event_code_to_word[think_event_id] = i  
        
        epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, preload=True, verbose='ERROR', baseline=(0, 0))
        X = epochs.get_data() 
        y_event_ids = epochs.events[:, 2]
        
        y_word = [event_code_to_word.get(label, -1) for label in y_event_ids]
        y_word = np.array(y_word)
        valid_indices = (y_word != -1)
        X = X[valid_indices]
        y_word = y_word[valid_indices]
        new_X = []
        new_y_word = []
        for i in range(X.shape[0]):
            for j in range(3):
                new_X.append(X[i, :, j * int(X.shape[2] / 3): (j + 1) * int(X.shape[2] / 3)])
                new_y_word.append(y_word[i]) 
        
        new_X = np.array(new_X)
        new_y_word = np.array(new_y_word)
        
        all_X.append(new_X)
        all_y_word.append(new_y_word)
    
    all_X = np.concatenate(all_X, axis=0)
    all_y_word = np.concatenate(all_y_word, axis=0)
    return all_X, all_y_word

def standardize_data(X):
    scaler = StandardScaler()
    num_samples, num_channels, num_timepoints = X.shape
    X = X.reshape(num_samples, -1)
    X = scaler.fit_transform(X)
    X = X.reshape(num_samples, num_channels, num_timepoints)
    return X

# 自定义数据集类
class EEGDataset(Dataset):
    def __init__(self, X, y_word):
        self.X = X.astype(np.float32)  
        self.y_word = y_word
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y_word[idx]

class EEGNetWithAttention(nn.Module):
    def __init__(self, num_classes_word, dropout=0.3):
        super(EEGNetWithAttention, self).__init__()
        self.firstconv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            nn.BatchNorm2d(16)
        )
        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(64, 1), stride=(1, 1), groups=16, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4)),
            nn.Dropout(dropout)
        )
        self.separableConv = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8)),
            nn.Dropout(dropout)
        )
        self.num_classes_word = num_classes_word
        self.fc = None

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension, shape: [N, 1, C, T]
        x = self.firstconv(x)
        x = self.depthwiseConv(x)
        x = self.separableConv(x)
        x = x.flatten(start_dim=1)  # Flatten except batch dimension
        
        # 动态初始化全连接层
        if self.fc is None:
            input_features = x.shape[1]
            self.fc = nn.Linear(input_features, self.num_classes_word).to(x.device)
        
        x = self.fc(x)  # 输出分类结果
        return x

# 迁移学习训练函数
def train_transfer_model(model, train_loader, criterion_word, optimizer, num_epochs):
    train_losses = []
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    early_stop_patience = 10
    early_stop_counter = 0
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        running_loss_word = 0.0
        correct_word = 0
        total = 0
        
        for inputs, labels_word in train_loader:
            inputs = inputs.float().to(device) 
            labels_word = labels_word.long().to(device)

            optimizer.zero_grad()
            outputs_word = model(inputs)
            loss_word = criterion_word(outputs_word, labels_word)
            loss_word.backward()
            optimizer.step()
            
            running_loss_word += loss_word.item() * inputs.size(0)
            _, predicted_word = torch.max(outputs_word.data, 1)
            total += labels_word.size(0)
            correct_word += (predicted_word == labels_word).sum().item()

        epoch_loss_word = running_loss_word / total
        epoch_accuracy_word = 100 * correct_word / total
        scheduler.step(epoch_loss_word)
        
        train_losses.append(epoch_loss_word)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss Word: {epoch_loss_word:.4f}, Accuracy Word: {epoch_accuracy_word:.2f}%')

        # Early stopping check
        if epoch_loss_word < best_loss:
            best_loss = epoch_loss_word
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print("Early stopping triggered.")
                break

    return train_losses

# 测试模型函数
def test_model(model, test_loader, criterion_word):
    model.eval()
    running_loss_word = 0.0
    correct_word = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels_word in test_loader:
            inputs = inputs.float().to(device)
            labels_word = labels_word.long().to(device)

            outputs_word = model(inputs)
            loss_word = criterion_word(outputs_word, labels_word)
            running_loss_word += loss_word.item() * inputs.size(0)
            _, predicted_word = torch.max(outputs_word.data, 1)
            total += labels_word.size(0)
            correct_word += (predicted_word == labels_word).sum().item()
            all_labels.extend(labels_word.cpu().numpy())
            all_predictions.extend(predicted_word.cpu().numpy())

    test_loss_word = running_loss_word / total
    test_accuracy_word = 100 * correct_word / total
    print(f'Test Loss: {test_loss_word:.4f}, Test Accuracy: {test_accuracy_word:.2f}%')
    print(classification_report(all_labels, all_predictions))
    print(confusion_matrix(all_labels, all_predictions))

if __name__ == "__main__":
    words = [
        '农民种菜', '厨师做饭', '祖父喝茶', '病人咳嗽', 
        '叛军投降', '孔雀开屏', '老牛耕地', '母鸡下蛋', 
        '蜻蜓点水', '螳螂捕蝉'
    ]
    set_files = ['./008C1.set', './008C2.set']   # 替换为新数据集的文件路径

    X, y_word = process_set_files(set_files, words, 0, 3)
    X = standardize_data(X)
    
    # 数据集划分
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X)):
        print(f'Fold {fold+1}/5')
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y_word[train_idx], y_word[test_idx]

        # 创建数据加载器
        train_dataset = EEGDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_dataset = EEGDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

        # 加载预训练模型
        eegnet = EEGNetWithAttention(num_classes_word=len(words)).to(device)
        pretrained_dict = torch.load('eegnet_pretrained.pth')
        model_dict = eegnet.state_dict()
        
        # 加载预训练参数，忽略分类层
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and 'fc' not in k}
        model_dict.update(pretrained_dict)
        eegnet.load_state_dict(model_dict)

        # 冻结前面的卷积层，解冻最后的卷积模块和分类层
        for name, param in eegnet.named_parameters():
            if 'separableConv' in name or 'fc' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        # 检查是否有需要训练的参数
        trainable_params = list(filter(lambda p: p.requires_grad, eegnet.parameters()))
        if len(trainable_params) == 0:
            raise ValueError("No trainable parameters found. Ensure at least one layer requires gradient.")

        # 定义损失函数和优化器
        criterion_word = nn.CrossEntropyLoss()
        optimizer = torch.optim.AdamW(trainable_params, lr=0.001, weight_decay=5e-4)

        # 迁移学习训练
        train_transfer_model(eegnet, train_loader, criterion_word, optimizer, num_epochs=100)

        # 测试模型
        test_model(eegnet, test_loader, criterion_word)

        print("迁移学习训练和测试完成")
        break  # 只进行一次折训练用于迁移学习


Reading c:\Users\clock\Desktop\hanzi\008C1.fdt
Reading 0 ... 903022  =      0.000 ...   903.022 secs...
文件读取成功！
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59']
Reading c:\Users\clock\Desktop\hanzi\008C2.fdt
Reading 0 ... 901822  =      0.000 ...   901.822 secs...


C:\Users\clock\AppData\Local\Temp\ipykernel_22980\2838680998.py:50: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()


文件读取成功！
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59']


C:\Users\clock\AppData\Local\Temp\ipykernel_22980\2838680998.py:50: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()


Fold 1/5
Epoch [1/100], Loss Word: 2.4022, Accuracy Word: 8.85%
Epoch [2/100], Loss Word: 2.3717, Accuracy Word: 10.59%
Epoch [3/100], Loss Word: 2.3308, Accuracy Word: 14.24%
Epoch [4/100], Loss Word: 2.2836, Accuracy Word: 13.89%
Epoch [5/100], Loss Word: 2.2589, Accuracy Word: 14.41%
Epoch [6/100], Loss Word: 2.2649, Accuracy Word: 14.76%
Epoch [7/100], Loss Word: 2.2208, Accuracy Word: 17.36%
Epoch [8/100], Loss Word: 2.1910, Accuracy Word: 19.10%
Epoch [9/100], Loss Word: 2.1615, Accuracy Word: 21.70%
Epoch [10/100], Loss Word: 2.1690, Accuracy Word: 20.31%
Epoch [11/100], Loss Word: 2.1545, Accuracy Word: 19.44%
Epoch [12/100], Loss Word: 2.1197, Accuracy Word: 22.57%
Epoch [13/100], Loss Word: 2.1115, Accuracy Word: 23.61%
Epoch [14/100], Loss Word: 2.0931, Accuracy Word: 25.87%
Epoch [15/100], Loss Word: 2.0775, Accuracy Word: 25.35%
Epoch [16/100], Loss Word: 2.0519, Accuracy Word: 30.03%
Epoch [17/100], Loss Word: 2.0347, Accuracy Word: 30.90%
Epoch [18/100], Loss Word: 2.030

In [16]:
import os
import mne
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def process_set_files(set_files, words, tmin, tmax):
    all_X = []
    all_y_word = []
    for set_file in set_files:
        try:
            raw = mne.io.read_raw_eeglab(set_file, preload=True)
            print("文件读取成功！")
        except Exception as e:
            print(f"读取文件时出错: {e}")
            exit()

        channels_to_drop = [ch for ch in ['VEOG', 'HEOG', 'Trigger'] if ch in raw.ch_names]
        raw.drop_channels(channels_to_drop)

        raw.filter(0.5, 80., fir_design='firwin')

        events, current_event_id_map = mne.events_from_annotations(raw)
        event_id = {}
        event_code_to_word = {}
        
        for i, word in enumerate(words):
            think_code = str(50 + i)  
            think_event_id = current_event_id_map.get(think_code)
            
            if think_event_id is not None:
                event_id[f'想_{word}'] = think_event_id
                event_code_to_word[think_event_id] = i  
        
        epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, preload=True, verbose='ERROR', baseline=(0, 0))
        X = epochs.get_data() 
        y_event_ids = epochs.events[:, 2]
        
        y_word = [event_code_to_word.get(label, -1) for label in y_event_ids]
        y_word = np.array(y_word)
        valid_indices = (y_word != -1)
        X = X[valid_indices]
        y_word = y_word[valid_indices]
        new_X = []
        new_y_word = []
        for i in range(X.shape[0]):
            for j in range(3):
                new_X.append(X[i, :, j * int(X.shape[2] / 3): (j + 1) * int(X.shape[2] / 3)])
                new_y_word.append(y_word[i]) 
        
        new_X = np.array(new_X)
        new_y_word = np.array(new_y_word)
        
        all_X.append(new_X)
        all_y_word.append(new_y_word)
    
    all_X = np.concatenate(all_X, axis=0)
    all_y_word = np.concatenate(all_y_word, axis=0)
    return all_X, all_y_word

def standardize_data(X):
    scaler = StandardScaler()
    num_samples, num_channels, num_timepoints = X.shape
    X = X.reshape(num_samples, -1)
    X = scaler.fit_transform(X)
    X = X.reshape(num_samples, num_channels, num_timepoints)
    return X

class EEGDataset(Dataset):
    def __init__(self, X, y_word):
        self.X = X.astype(np.float32)  
        self.y_word = y_word
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y_word[idx]

class EEGNetWithAttention(nn.Module):
    def __init__(self, num_classes_word, dropout=0.3):
        super(EEGNetWithAttention, self).__init__()
        self.firstconv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            nn.BatchNorm2d(16)
        )
        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(64, 1), stride=(1, 1), groups=16, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4)),
            nn.Dropout(dropout)
        )
        self.separableConv = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8)),
            nn.Dropout(dropout)
        )
        self.num_classes_word = num_classes_word
        self.fc = None

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.firstconv(x)
        x = self.depthwiseConv(x)
        x = self.separableConv(x)
        x = x.flatten(start_dim=1)
        
        if self.fc is None:
            input_features = x.shape[1]
            self.fc = nn.Linear(input_features, self.num_classes_word).to(x.device)
        
        x = self.fc(x)
        return x

def train_transfer_model(model, train_loader, criterion_word, optimizer, num_epochs):
    train_losses = []
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    early_stop_patience = 10
    early_stop_counter = 0
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        running_loss_word = 0.0
        correct_word = 0
        total = 0
        
        for inputs, labels_word in train_loader:
            inputs = inputs.float().to(device) 
            labels_word = labels_word.long().to(device)

            optimizer.zero_grad()
            outputs_word = model(inputs)
            loss_word = criterion_word(outputs_word, labels_word)
            loss_word.backward()
            optimizer.step()
            
            running_loss_word += loss_word.item() * inputs.size(0)
            _, predicted_word = torch.max(outputs_word.data, 1)
            total += labels_word.size(0)
            correct_word += (predicted_word == labels_word).sum().item()

        epoch_loss_word = running_loss_word / total
        epoch_accuracy_word = 100 * correct_word / total
        scheduler.step(epoch_loss_word)
        
        train_losses.append(epoch_loss_word)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss Word: {epoch_loss_word:.4f}, Accuracy Word: {epoch_accuracy_word:.2f}%')

        if epoch_loss_word < best_loss:
            best_loss = epoch_loss_word
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print("Early stopping triggered.")
                break

    return train_losses

def test_model(model, test_loader, criterion_word):
    model.eval()
    running_loss_word = 0.0
    correct_word = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels_word in test_loader:
            inputs = inputs.float().to(device)
            labels_word = labels_word.long().to(device)

            outputs_word = model(inputs)
            loss_word = criterion_word(outputs_word, labels_word)
            running_loss_word += loss_word.item() * inputs.size(0)
            _, predicted_word = torch.max(outputs_word.data, 1)
            total += labels_word.size(0)
            correct_word += (predicted_word == labels_word).sum().item()
            all_labels.extend(labels_word.cpu().numpy())
            all_predictions.extend(predicted_word.cpu().numpy())

    test_loss_word = running_loss_word / total
    test_accuracy_word = 100 * correct_word / total
    print(f'Test Loss: {test_loss_word:.4f}, Test Accuracy: {test_accuracy_word:.2f}%')
    print(classification_report(all_labels, all_predictions))
    print(confusion_matrix(all_labels, all_predictions))

if __name__ == "__main__":
    words = [
        '农民种菜', '厨师做饭', '祖父喝茶', '病人咳嗽', 
        '叛军投降', '孔雀开屏', '老牛耕地', '母鸡下蛋', 
        '蜻蜓点水', '螳螂捕蝉'
    ]
    set_files = ['./008C1.set', './008C2.set']

    X, y_word = process_set_files(set_files, words, 0, 3)
    X = standardize_data(X)
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X)):
        print(f'Fold {fold+1}/5')
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y_word[train_idx], y_word[test_idx]

        train_dataset = EEGDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_dataset = EEGDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

        eegnet = EEGNetWithAttention(num_classes_word=len(words)).to(device)
        pretrained_dict = torch.load('eegnet_pretrained.pth')
        model_dict = eegnet.state_dict()
        
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and 'fc' not in k}
        model_dict.update(pretrained_dict)
        eegnet.load_state_dict(model_dict)

        for name, param in eegnet.named_parameters():
            if 'separableConv' in name or 'fc' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        trainable_params = list(filter(lambda p: p.requires_grad, eegnet.parameters()))
        if len(trainable_params) == 0:
            raise ValueError("No trainable parameters found. Ensure at least one layer requires gradient.")

        criterion_word = nn.CrossEntropyLoss()
        optimizer = torch.optim.AdamW(trainable_params, lr=0.001, weight_decay=5e-4)

        train_transfer_model(eegnet, train_loader, criterion_word, optimizer, num_epochs=200)

        test_model(eegnet, test_loader, criterion_word)

        print("迁移学习训练和测试完成")
        break


Reading c:\Users\clock\Desktop\hanzi\008C1.fdt
Reading 0 ... 903022  =      0.000 ...   903.022 secs...
文件读取成功！
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59']
Reading c:\Users\clock\Desktop\hanzi\008C2.fdt
Reading 0 ... 901822  =      0.000 ...   901.822 secs...


C:\Users\clock\AppData\Local\Temp\ipykernel_22980\1251669060.py:44: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()


文件读取成功！
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59']


C:\Users\clock\AppData\Local\Temp\ipykernel_22980\1251669060.py:44: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()


Fold 1/5
Epoch [1/200], Loss Word: 2.4357, Accuracy Word: 9.38%
Epoch [2/200], Loss Word: 2.3792, Accuracy Word: 10.59%
Epoch [3/200], Loss Word: 2.3439, Accuracy Word: 10.07%
Epoch [4/200], Loss Word: 2.3087, Accuracy Word: 12.15%
Epoch [5/200], Loss Word: 2.2869, Accuracy Word: 12.67%
Epoch [6/200], Loss Word: 2.2635, Accuracy Word: 17.01%
Epoch [7/200], Loss Word: 2.2305, Accuracy Word: 18.40%
Epoch [8/200], Loss Word: 2.2278, Accuracy Word: 17.01%
Epoch [9/200], Loss Word: 2.1892, Accuracy Word: 18.23%
Epoch [10/200], Loss Word: 2.1899, Accuracy Word: 18.75%
Epoch [11/200], Loss Word: 2.1555, Accuracy Word: 20.14%
Epoch [12/200], Loss Word: 2.1397, Accuracy Word: 22.57%
Epoch [13/200], Loss Word: 2.1489, Accuracy Word: 23.26%
Epoch [14/200], Loss Word: 2.1050, Accuracy Word: 24.31%
Epoch [15/200], Loss Word: 2.1180, Accuracy Word: 21.18%
Epoch [16/200], Loss Word: 2.0628, Accuracy Word: 28.47%
Epoch [17/200], Loss Word: 2.0642, Accuracy Word: 28.12%
Epoch [18/200], Loss Word: 2.038